In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import TensorBoard
from time import time
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

trainDir = "E:\\594_data\\imbalanced_Pixelwise_NN_Data\\train"
validDir = "E:\\594_data\\imbalanced_Pixelwise_NN_Data\\valid"

c:\users\tyler\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def f2_score(y_true, y_pred):
    y_true = tf.cast(y_true, "int32")
    y_pred = tf.cast(tf.round(y_pred), "int32") # implicit 0.5 threshold via tf.round
    y_correct = y_true * y_pred
    sum_true = tf.reduce_sum(y_true, axis=1)
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    sum_correct = tf.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 5 * precision * recall / (4 * precision + recall)
    f_score = tf.where(tf.is_nan(f_score), tf.zeros_like(f_score), f_score)
    return tf.reduce_mean(f_score)

In [3]:
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        data_format='channels_first',
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255,         
                                  data_format='channels_first'
                                 )

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        trainDir,  # this is the target directory
        target_size=(65, 65),  # all images will be resized
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

label_map = (train_generator.class_indices)
print(label_map)

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        validDir,
        target_size=(65, 65),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='binary')

Found 264301 images belonging to 2 classes.
{'hasCell': 0, 'noCell': 1}
Found 35699 images belonging to 2 classes.


In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(1,65,65 ), data_format='channels_first'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


checkpoint = ModelCheckpoint("weights.best.imbalanced.hdf5", verbose=1, monitor='val_binary_accuracy',save_best_only=True, mode='auto')

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

class_weight = {0: 1000.,
                1: 1.}

model.fit_generator(
        train_generator,
        steps_per_epoch=264301 // batch_size,
        epochs=100,
        validation_data=validation_generator,
        validation_steps=35699 // batch_size,
        verbose=1, 
        callbacks=[checkpoint,tensorboard])

Epoch 1/100
8258/8259 [============================>.] - ETA: 0s - loss: 0.0669 - binary_accuracy: 0.9956
Epoch 00001: val_binary_accuracy improved from -inf to 0.99535, saving model to weights.best.imbalanced.hdf5
8259/8259 [==============================] - 4040s 489ms/step - loss: 0.0668 - binary_accuracy: 0.9956 - val_loss: 0.0742 - val_binary_accuracy: 0.9953
Epoch 2/100
5028/8259 [=================>............] - ETA: 1:10 - loss: 0.0659 - binary_accuracy: 0.9959

KeyboardInterrupt: 